### How to install C++ kernel in Jupyter? 
1. Install miniconda
2. Create a new environment: ```conda create -n cpp_env```
<!-- (3. Initialise the environment: ```conda init```) -->
4. Activate the environment: ```conda activate cpp_env```
5. Install xeus-cling: ```conda install jupyter xeus-cling -c conda-forge```
<!--6. Launch Jupyter: ```jupyter notebook --no-browser```
7. Change kernel, select another kernel, existing jupyter server and copy-paste (if needed) the url provided in the terminal -->
8. Change kernel and select a C++ kernel

### Black-Scholes
$$C(S_t, t) = N(d_+)S_t e^{-q(T - t)} - N(d_-)Ke^{-r(T - t)}$$
$$d_+ = \frac{1}{\sigma\sqrt{T - t}}\left[\ln\left(\frac{S_t}{K}\right) + \left(r-q + \frac{\sigma^2}{2}\right)(T - t)\right] $$
$$     d_- = d_+ - \sigma\sqrt{T - t} $$

In [1]:
#include <cmath>
#include <iostream>
#include <string>
#include <algorithm>
#include <stdexcept>
#include <random>
using namespace std;

In [2]:
double normal_cdf_riemann(double b){
    double a=-50; //Limit of integration

    int n=100000; // Number of subintervals
    double dx = (b-a)/n;
    double s=0.0;
    double x=a;

    for (int i=0; i<n; i++){
        s+= exp(-pow(x,2)/2)*dx;
        x+=dx; 
    }
    return s/sqrt(2*M_PI);;
}

In [3]:
double Black_Scholes(string call_put, double sigma, double K, double T, double t, double St, double r, double q){
    double d_plus = (log(St/K)+(r-q+pow(sigma,2)/2)*(T-t))/(sigma*sqrt(T-t));
    double d_minus = d_plus - sigma*sqrt(T-t);
    if(call_put=="call"){
        return normal_cdf_riemann(d_plus)*St*exp(-q*(T-t))- normal_cdf_riemann(d_minus)*K*exp(-r*(T-t));;
    }
    else if (call_put=="put"){
        return -normal_cdf_riemann(-d_plus)*St*exp(-q*(T-t))+ normal_cdf_riemann(-d_minus)*K*exp(-r*(T-t));;
    }
    else{
        throw runtime_error("Please choose between call and put");
        //return -1.;
    }
}

In [4]:
double St=100;
double K=100;
double T=1.;
double t=0;
double r=.05;
double q=0.5;
double sigma=.2;
cout << "Black-Scholes call price: "+to_string(Black_Scholes("call", sigma, K, T, t, St, r, q)) << endl;
cout << "Black-Scholes put price: "+to_string(Black_Scholes("put", sigma, K, T, t, St, r, q)) << endl;


Black-Scholes call price: 0.064067
Black-Scholes put price: 34.533943


### Monte-Carlo simulation

In [2]:
double Monte_Carlo(string call_put, double sigma, double K, double T, double t, double St, double r, double q){  
    default_random_engine generator; // Create a generator
    normal_distribution<double> distribution(0,1); //Specify the distribution
    int n=10; // Number of time points
    double dt=(T-t)/(n-1);
    int N=50000; //Number of paths
    double S[N][n];
    for (int i=0; i<N; i++){
      S[i][0]=St; //Initialise all the paths at St
    }

    for (int i = 0; i < N; i++) {
      for (int j=1; j<n; j++){
        S[i][j] = S[i][j-1]*exp((r-q-pow(sigma,2)/2)*dt+sigma*sqrt(dt)*distribution(generator));
      }
    }

    double payoff_sum;
    if (call_put=="call"){
        for (int i=0; i<N; i++){
            payoff_sum += max(S[i][n-1]-K,0.);
        }
    }
    else if(call_put=="put"){
        for (int i=0; i<N; i++){
            payoff_sum += max(K-S[i][n-1],0.);
        }
    }
    else{
        throw runtime_error("Please choose between call and put");
    }
    return payoff_sum/N*exp(-r*(T-t));
}

In [3]:
double St=100;
double K=100;
double T=1.;
double t=0;
double r=.05;
double q=0.;
double sigma=.2;

cout << "Monte-Carlo European call price: "+to_string(Monte_Carlo("call", sigma, K, T, t, St, r, q)) << endl;
cout << "Monte-Carlo European put price: "+to_string(Monte_Carlo("put", sigma, K, T, t, St, r, q)) << endl;

Monte-Carlo European call price: 10.375356
Monte-Carlo European put price: 5.589298


Monte-Carlo American call price: 15.436859
Monte-Carlo American put price: 9.609382


### Tree

In [9]:
double Tree(string call_put, string am_eu, double sigma, double K, double T, double t, double St, double r, double q){  
    int n=150; // Number of time periods
    double dt=(T-t)/n;
    double S[n][n+1];
    double U=exp(sigma*sqrt(dt));
    double D=1/U;

    S[0][0]=St;
    
    for (int i = 0; i<n-1; i++) {
      for (int j=0; j<i+1; j++){ //the tree recombines so there is exactly i+1 nodes after the ith time period
        S[i+1][j] = S[i][j]*U; //the three recombines so all the i+1 nodes are created by taking the up of all previous nodes 
      }
      S[i+1][i+1]=S[i][i]*D; // except the last node that must be created by taking the down of the last i node
    }

    double prices[n][n+1];
    if(call_put=="call"){
      for (int j=0; j<n+1; j++){ 
        prices[n-1][j]=max(S[n-1][j]-K,0.);
      }
    }
    else if(call_put=="put"){
      for (int j=0; j<n+1; j++){ 
        prices[n-1][j]=max(K-S[n-1][j],0.);
      }
    }
    else{
        throw runtime_error("Please choose between call and put");
    }

    double p=(exp((r-q)*dt)-D)/(U-D);
    if(am_eu=="European"){
      for (int i = n-2; i>=0; i--) {
        for (int j=0; j<i+1; j++){ 
          prices[i][j] = (p*prices[i+1][j]+(1-p)*prices[i+1][j+1])*exp(-r*dt);
        }
      }
    }
    else if(am_eu=="American"){
      if(call_put=="call"){
        for (int i = n-2; i>=0; i--) {
          for (int j=0; j<i+1; j++){ 
            prices[i][j] = max((p*prices[i+1][j]+(1-p)*prices[i+1][j+1])*exp(-r*dt), S[i][j]-K);
          }
        }
      }
      else if(call_put=="put"){
        for (int i = n-2; i>=0; i--) {
          for (int j=0; j<i+1; j++){ 
            prices[i][j] = max((p*prices[i+1][j]+(1-p)*prices[i+1][j+1])*exp(-r*dt), K-S[i][j]);
          }
        }
      }
    }
    else
    {
        throw runtime_error("Please choose American or European.");
    }

    return prices[0][0];;
}

In [11]:
double St=100;
double K=100;
double T=1.;
double t=0;
double r=.05;
double q=0.5;
double sigma=.2;

cout << "Binomial European tree call price: "+to_string(Tree("call", "European", sigma, K, T, t, St, r, q)) << endl;
cout << "Binomial European tree put price: "+to_string(Tree("put", "European", sigma, K, T, t, St, r, q)) << endl;

cout << "Binomial American tree call price: "+to_string(Tree("call", "American", sigma, K, T, t, St, r, q)) << endl;
cout << "Binomial American tree put price: "+to_string(Tree("put", "American", sigma, K, T, t, St, r, q)) << endl;

Binomial European tree call price: 0.062082
Binomial European tree put price: 34.361158
Binomial American tree call price: 1.556477
Binomial American tree put price: 34.361158


### PDE and finite difference methods

$$\frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} +  (r-q)S\frac{\partial V}{\partial S} -  rV = 0$$
$$\Theta + \frac{1}{2}\sigma^2 S^2 \Gamma +  (r-q)S \Delta -  rV = 0$$
We can discretize the Greeks (I choose to use the explicit method, ie backward approximation for $\Theta$, central approximation for $\Delta$ and standard approximation for $\Gamma$):
$$\Theta =\frac{\partial V}{\partial t} = \frac{V(t_i,S_j)-V(t_{i-1},S_j)}{\delta t}$$
$$\Gamma =\frac{\partial^2 V}{\partial S^2}= \frac{V(t_i,S_{j+1})-2V(t_i,S_j)+V(t_i,S_{j-1})}{(\delta S)^2}$$
$$ \Delta =\frac{\partial V}{\partial S}= \frac{V(t_i,S_{j+1})-V(t_i,S_{j-1})}{2\delta S}$$
Thus, we have:
$$V(t_{i-1},S_j)=(1-r \delta t) V(t_i,S_j)+\frac{1}{2}\sigma^2 \delta t S_j^2 \frac{V(t_i,S_{j+1})+2V(t_i,S_j)+V(t_i,S_{j-1})}{(\delta S)^2} + (r-q) \delta t S_j \frac{V(t_i,S_{j+1})-V(t_i,S_{j-1})}{2\delta S}$$
$$V(t_{i-1},S_j)= a_j V(t_i,S_{j-1}) + b_j V(t_i, S_j) + c_j V(t_i, S_{j+1})$$
with $a_j=\frac{1}{2} \delta t (\sigma^2 j^2-(r-q)j) $, $b_j=1-\delta t (\sigma^2 j^2 +r)$ and $c_j=\frac{1}{2} \delta t (\sigma^2 j^2+(r-q)j)$ using $S_j=j \delta S$.

We set boundaries conditions:
$$ \forall i, V(t_i,0) = 0 $$
$$ \forall i, V(t_i, S_j) \underset{S_j \rightarrow \infty}{\sim} S_j - K e^{-r(T-t_i)}$$
$$ \forall j, V(T, S_j) = \max(S_j - K, 0)$$

In [1]:
#include <cmath>
#include <iostream>
#include <string>
#include <algorithm>
#include <stdexcept>
#include <random>
using namespace std;

In [2]:
double PDE(string call_put, string am_eu, double sigma, double K, double T, double St, double r, double q){  
    //double S_min =0.;
    //double S_max= St*(1+8*sigma*sqrt(T)); //good maximum value for a lognormal distribution
    double S_max= St*(1+2*sigma*sqrt(T)); 
    int n_t=1000; // Number of time points
    int n_S=50; // Number of price points
    double dt=T/(n_t-1);
    double dS=(S_max)/(n_S-1);
    double V[n_t][n_S];

    if(dt*sigma*sigma*n_S*n_S>1){
        cout << "Change the parameters in order to assure the stability." << endl;
    }

    if(call_put=="call"){
        for(int i=0; i<n_t;i++){
            V[i][0] = 0.;
            V[i][n_S-1] = S_max - K*exp(-r*(T-i*dt));
        }
        for(int j=0; j<n_S;j++){
            V[n_t-1][j] = max(j*dS-K,0.);
        }
    }
    else if (call_put=="put"){
        for(int i=0; i<n_t;i++){
            V[i][0] = K*exp(-r*(T-i*dt));
            V[i][n_S-1] = 0.;
        }
        for(int j=0; j<n_S;j++){
            V[n_t-1][j] = max(K-j*dS,0.);
        }
    }
    else{
        throw runtime_error("Please choose between call and put");
    }
    
    double a,b,c;
    if(am_eu=="European"){
        for(int i=n_t-1; i>0;i--){
            for(int j=1; j<n_S-1;j++){
                a=.5*dt*(sigma*sigma*j*j-(r-q)*j);
                b=1-dt*(sigma*sigma*j*j+r);
                c=.5*dt*(sigma*sigma*j*j+(r-q)*j);
                V[i-1][j] = a*V[i][j-1]+b*V[i][j]+c*V[i][j+1];
            }
        }
    }
    else if(am_eu=="American"){
        if(call_put=="call"){
            for(int i=n_t-1; i>0;i--){
                for(int j=1; j<n_S-1;j++){
                    a=.5*dt*(sigma*sigma*j*j-(r-q)*j);
                    b=1-dt*(sigma*sigma*j*j+r);
                    c=.5*dt*(sigma*sigma*j*j+(r-q)*j);
                    V[i-1][j] = max(a*V[i][j-1]+b*V[i][j]+c*V[i][j+1], j*dS-K);
                }
            }
        }
        else if(call_put=="put"){
            for(int i=n_t-1; i>0;i--){
                for(int j=1; j<n_S-1;j++){
                    a=.5*dt*(sigma*sigma*j*j-(r-q)*j);
                    b=1-dt*(sigma*sigma*j*j+r);
                    c=.5*dt*(sigma*sigma*j*j+(r-q)*j);
                    V[i-1][j] = max(a*V[i][j-1]+b*V[i][j]+c*V[i][j+1], K-j*dS);
                }
            }
        }
    }
    else{
        throw runtime_error("Please choose between American and European");
    }
    return V[0][int(round(St/dS))];;
}

In [3]:
double St=100;
double K=100;
double T=1.;
double t=0;
double r=.05;
double q=0.5;
double sigma=.2;
cout << "PDE European call price: "+to_string(PDE("call", "European", sigma, K, T, St, r, q)) << endl;
cout << "PDE European put price: "+to_string(PDE("put", "European", sigma, K, T, St, r, q)) << endl;
cout << "PDE American call price: "+to_string(PDE("call", "American", sigma, K, T, St, r, q)) << endl;
cout << "PDE American put price: "+to_string(PDE("put", "American", sigma, K, T, St, r, q)) << endl;

PDE European call price: 0.061975
PDE European put price: 34.533300
PDE American call price: 1.461421
PDE American put price: 34.533300


try on MC for am options:
double Monte_Carlo(string call_put, string am_eu, double sigma, double K, double T, double t, double St, double r, double q){  
    default_random_engine generator; // Create a generator
    normal_distribution<double> distribution(0,1); //Specify the distribution
    int n=10; // Number of time points
    double dt=(T-t)/(n-1);
    int N=50000; //Number of paths
    double S[N][n];
    for (int i=0; i<N; i++){
      S[i][0]=St; //Initialise all the paths at St
    }
    

    for (int i = 0; i < N; i++) {
      for (int j=1; j<n; j++){
        S[i][j] = S[i][j-1]*exp((r-q-pow(sigma,2)/2)*dt+sigma*sqrt(dt)*distribution(generator));
      }
    }

    double option_price=-1.;
    if (am_eu=="European")
    {
        double payoff_sum;
        if (call_put=="call"){
        for (int i=0; i<N; i++){
            payoff_sum += max(S[i][n-1]-K,0.);
        }
        }
        else if(call_put=="put"){
        for (int i=0; i<N; i++){
            payoff_sum += max(K-S[i][n-1],0.);
        }
        }
        else{
            throw runtime_error("Please choose between call and put");
        }
        option_price=payoff_sum/N*exp(-r*(T-t));
    }
    else if (am_eu=="American")
    {
        double prices[N][n];
        if (call_put=="call"){
          for (int i=0; i<N; i++){
              prices[i][n-1] = max(S[i][n-1]-K,0.);
          }
          for (int i = 0; i<N; i++) {
              for (int j=n-2; j>=0; j--){ 
                  prices[i][j] = max(S[i][j]-K, prices[i][j+1]*exp(-r*dt));
                  //prices[i][j] = max(S[i][j]-K, prices[i][j+1]*exp(-(r-q)*dt));
              }
          }
        }
        else if(call_put=="put"){
          for (int i=0; i<N; i++){
              prices[i][n-1] = max(K-S[i][n-1],0.);
          }
          for (int i = 0; i<N; i++) {
              for (int j=n-2; j>=0; j--){ 
                  prices[i][j] = max(K-S[i][j], prices[i][j+1]*exp(-r*dt));
                  //prices[i][j] = max(S[i][j]-K, prices[i][j+1]*exp(-(r-q)*dt));
              }
          }
        }
        else{
            throw runtime_error("Please choose between call and put");
        }

        double price_sum=0;
        for (int i=0; i<N; i++){
            price_sum += prices[i][0];
        }
        option_price=price_sum/N;
    }
    else
    {
        throw runtime_error("Please choose American or European.");
    }
    return option_price;;
}
